# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys

#Passo 1: Pegando cotação dólar
#Abrir o google Chrome primeiramente
navegador= webdriver.Chrome()
#Entrar no site do google
navegador.get('https://www.google.com/')
#Quero encontrar um elemento pra clicar no navegador
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar= navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


#Passo 2: Pegando cotacao euro 
navegador.get('https://www.google.com/')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro= navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


#Passo 3: Pegando cotacao ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro= navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')

navegador.quit()

#cotação do ouro está com , e nao com o ., então vamos mudar:
cotacao_ouro= cotacao_ouro.replace(',','.')
print('Cotação Dólar é:',cotacao_dolar)
print('Cotação Euro é:', cotacao_euro)
print('Cotação Ouro é:',cotacao_ouro)

Cotação Dólar é: 5.1354999999999995
Cotação Euro é: 5.57047685
Cotação Ouro é: 327.27


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
#Passo 4: Importar a base de dados 
import pandas as pd 

tabela= pd.read_excel('Produtos.xlsx')
display(tabela)



,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [3]:
#Passo 5: Recalcular os preços 
# atualizar a cotação
tabela.loc[tabela['Moeda']== 'Dólar', 'Cotação']= float(cotacao_dolar)
tabela.loc[tabela['Moeda']== 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda']== 'Ouro','Cotação'] = float(cotacao_ouro)

# recalcular os preços 
# preço de compra = preço original * cotação
tabela['Preço de Compra']= tabela['Preço Original'] * tabela['Cotação']

# preço de venda = preço de compra * margem 

tabela['Preço de Venda']= tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.135500,5135.448645,1.40,7189.628103
1,Carro Renault,4500.00,Euro,5.570477,25067.145825,2.00,50134.291650
2,Notebook Dell,899.99,Dólar,5.135500,4621.898645,1.70,7857.227696
3,IPhone,799.00,Dólar,5.135500,4103.264500,1.70,6975.549650
4,Carro Fiat,3000.00,Euro,5.570477,16711.430550,1.90,31751.718045
5,Celular Xiaomi,480.48,Dólar,5.135500,2467.505040,2.00,4935.010080
6,Joia 20g,20.00,Ouro,327.270000,6545.400000,1.15,7527.210000


### Agora vamos exportar a nova base de preços atualizada

In [9]:
#Passo 6: Exportar a nova base de dados
#para o índice do pandas nao ir junto para o arquivo, colocar index=False
tabela.to_excel('Produtos novos222.xlsx',index=False)